# Handling Images

In this notebook you will:

* Load image data into a table and discover that that is not so nice to work with.
* Load image data lazily using the ``.data()`` method.
* Perform streaming computations on the lazily-loaded data.
* Access multiple columns of data lazily using the ``.events()`` method.

Recommended Prerequisites:

* [Process Tabular Data with Pandas](./Process%20Tabular%20Data%20with%20Pandas.ipynb)

## Configuration

In [ ]:
# Runs EPICS IOC(s) with simulated hardware in leiu of actual motors, detectors.
!supervisor/start_supervisor.sh

In [ ]:
%run scripts/beamline_configuration.py
I.kind = 'normal'  # We do not wanted the beam current to be the 'hinted' value in this notebook.

## Array data in Theory

We can read more than just scalar data using Bluesky.  For handling
non-scalar data (such as from imaging detectors or MCAs) we do not
directly store the data in the databroker, instead we store _pointer_
to where the data is and how to access it.  This design allows us to buffer the user from such mundane details as what the filename and format of the underlying data is.  
This is also motivated in part by performance concerns both at the database level, you do not want to put gigabytes of binary data into a database, and at the collection level, we do not want to put Bluesky between detectors and disk. 

In short, these _pointers_ allow *DataBroker* to do the file I/O work of opening and extracting data from disk and returns to you numpy arrays.  For more [details about how this works](https://nsls-ii.github.io/databroker/assets.html) see the DataBroker documentation.


## Array data in practice

Lets take a single frame of a low signal-to-noise detector to see how this works

In [ ]:
RE(mv(spot.exp, .005))  # low exposure time so we will get significant noise
RE(mv(mtr_spotx, 0, mtr_spoty, 0))  # set position to dead center

RE(count([spot, I], num=1))
h_one = db[-1]

If we look at this with `table`:

In [ ]:
h_one.table()

We see a uid string in the column where we expect our image to be!  Taking a look at the `'spot_img'`'s entry in the descriptor

In [ ]:
h_one.descriptors[0]['data_keys']['spot_img']

we see that the expeted shape is `(480, 640)` and `'external'` key indicates that these strings are keys to trade to the *DataBroker* for the actual data. To ask
*DataBroker* to fetch the image data from disk we can pass the optional kwarg `fill`
to `table`

In [ ]:
h_one.table(fill=True)

In [ ]:
h_one.table(fill=True)['spot_img'][1]

but it is embedded in a Pandas data frame.  The `Header` has the
``data`` method which pulls out one column of the stream (and defaults to
`fill=True`).

The object returned by `h.data` is a *generator* which will lazily return
one value at a time.  To grab just the first image we can use `next`

In [ ]:
im = next(h_one.data('spot_img'))
im

In [ ]:
fig, ax = plt.subplots()
im_artist = ax.imshow(im)
fig.colorbar(im_artist)

You have already been working with *generator* functions as under-the-hood the built in  *BlueSky* plans are all generators.  For more details on generators see [the BlueSky appendix](https://nsls-ii.github.io/bluesky/appendix.html) or google ["James Powell generators youtube"](https://www.google.com/search?q=james+powell+generators+youtube).

If we know there is exactly 1 image we can unpack it like:

In [ ]:
im, = h_one.data('spot_img')

To play with generators a bit more, lets take some data with multiple frames.

In [ ]:
RE(count([spot, I], num=5))
h_few = db[-1]

If there is more than one we can use `list`, `np.stack`, or
Python's "generalize unpacking" to pull all of the images

In [ ]:
im_list = list(h_few.data('spot_img'))
im_stack = np.stack(h_few.data('spot_img'))
im1, *rest = h_few.data('spot_img')

We can also iterate through all of them with a `for` loop


In [ ]:
for j, im in enumerate(h_few.data('spot_img')):
    print(f'frame {j} has max {im.max()}')

This has the nice feature that there is only ever 1 frame in memory at a time.  For these 5 small images, this is not a huge issue, but this technique can allow you to process data significantly bigger than your available memory.

## Intermission

Take a few minutes to play the the 4 ways to get all of the images out of the generator

## Summing Images

Now lets take a bunch of images to improve the statistics!


In [ ]:
RE(count([spot, I], num=150))
h_lots = db[-1]

We can again use ``next`` to peek at the first image *without loading the rest of it*.

In [ ]:
im = next(h_lots.data('spot_img'))
fig, ax = plt.subplots()
im_artist = ax.imshow(im)
fig.colorbar(im_artist)

We can then grab the whole stack, average them to one image and
display the result:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
im_stack = np.stack(h_lots.data('spot_img'))

vmin = im_stack.min()
vmax = im_stack.max()

im1 = ax1.imshow(im_stack[0], vmin=vmin, vmax=vmax)
im2 = ax2.imshow(im_stack.mean(axis=0), vmin=vmin, vmax=vmax)

ax1.set_title('1 frame')
ax2.set_title('mean of stack')

## Lazy access over more than one column

`data` can pull out any column.  If you want to access both the beam current and the image you could access them separately and use ``zip`` to loop over them together:

In [ ]:
out = np.zeros((480, 640))
j = 0
for cur, im in zip(h_lots.data('I'), h_lots.data('spot_img')):
    out += im / cur  # cumulative sum of image pixel data
    j += 1  # count total number of images

out /= j  # i.e. convert the sum to a mean

However, if you need to access more than one key, it may be better to
use the *Event* documents directly.

In [ ]:
out = np.zeros((480, 640))
j = 0
for event in h_lots.events(fill=True):
    im = event['data']['spot_img']
    cur = event['data']['I']
    out += im / cur  # cumulative sum of image pixel data
    j += 1  # count total number of images

out /= j  # i.e. convert the sum to a mean

## Scanning with the spot detector

In [ ]:
RE(mv(spot.exp, 5))  # Set exposure time higher to get bettter signal-to-noise.

Lets take a look at how the spot changes as we scan the motors `mtr_spotx` and `mtr_spoty`:

In [ ]:
# be explict about the source of the scan and the uid
grid_uid, = RE(bp.grid_scan([spot, I], mtr_spotx, -100, 100, 5, mtr_spoty, -100, 100, 5, False))  
h_grid = db[grid_uid]

The table and live-plot showed us how the total intensity changed as a function of position, but we can dig into the events and look at 

In [ ]:
fig, ax_arr = plt.subplots(5, 5)
im_arts = []
d_min = np.inf
d_max = -np.inf
for ax, event in zip(ax_arr.ravel(), h_grid.events(fill=True)):
    data = event['data']
    ax.axis('off')
    im_arts.append(ax.imshow(data['spot_img']))
    d_min = min(d_min, data['spot_img'].min())
    d_max = max(d_max, data['spot_img'].max())
    ax.set_title(f'({data["motor_spotx"]}, {data["motor_spoty"]})', size='xx-small', pad=2)

# set the same color scale on all of the images
for art in im_arts:
    art.set_clim(d_min, d_max)

            

Which shows us that the spot changes intensity and moves around.

### Mapping the intensity

Now lets map out the intensity a bit more systematically

In [ ]:
RE(spiral_fermat([I, spot], mtr_spotx, mtr_spoty, 0, 0, 75, 75, 3, 1))
h_spiral = db[-1]

The live plot shows us a scatter of the x-y positions of the motor colored by the `'spot_roi'` which shows the overall drop off we expect as a function of position.  However, some of the structure in it maybe due to the variation in the beam current.  To investigate this lets make contour plots of the un-normalized and normalized data.

In [ ]:
def tricontour_helper(df, ax, title):
    """
    Plot a filled triangular contour plot of unstructured (ungridded) data.
    """
    ax.tricontour(df['x'], df['y'], df['color'], 10, linewidths=0.5, colors='k')
    t = ax.tricontourf(df['x'], df['y'], df['color'], 10)
    ax.set_title(title)
    ax.set_aspect('equal')

Fist, get a dataframe of the (x, y) positions and an (rough) proxy for the summed intensity in the peak.

In [ ]:
import pandas as pd

rows = []
for event in h_spiral.events(fill=True):
    data = event['data']  # just to save typing below...
    im = data['spot_img']

    rows.append({'x': data['motor_spotx'],
                 'y': data['motor_spoty'],
                 'color': np.mean(im[im > 1000])})

df = pd.DataFrame(rows)

Netx, get a dataframe of the (x, y) positions and an (rough) proxy for the summed intensity in the peak normalized by the measured beam current.

In [ ]:
import pandas as pd

rows = []
for event in h_spiral.events(fill=True):
    data = event['data']
    im = data['spot_img']

    rows.append({'x': data['motor_spotx'],
                 'y': data['motor_spoty'],
                 'color': np.mean(im[im > 1000]) / data['I']})

df_normed = pd.DataFrame(rows)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
tricontour_helper(df, ax1, 'un-normalized')
tricontour_helper(df_normed, ax2, 'normalized')